In [2]:
import pandas as pd
import json
from geopy.distance import vincenty

In [3]:
#pd.set_option('display.max_rows', len(df))

In [4]:
routes = {'k4':1,'g3':2}

In [72]:
name = "_k4_june_14"

In [ ]:
#Convert hours into minute

In [73]:
def get_hour_min(x):
    from datetime import datetime
    date_obj = datetime.strptime(x, '%H:%M:%S')
    
    #date = date_obj.strftime("%H:%M")
    return date_obj.hour * 60 +date_obj.minute

In [ ]:
#remove duplicates record for same bus stop and bus

In [74]:
def remove_duplicates(df):
    unique_data = pd.DataFrame()
    bus_stop = df.iloc[0]['bus_stop']
    bus = df.iloc[0]['device_id']
    #time = df.iloc[0]['date_time']
    unique_data = unique_data.append( df.iloc[0])
    for index,row in df.iterrows():
        if(bus_stop==row['bus_stop'] and bus==row['device_id']):
            continue
        else:
            unique_data = unique_data.append(row)
            bus_stop = row['bus_stop']
            bus = row['device_id']
            #time = row['date_time']
    return unique_data

In [1]:
#separate data into into two group (up and down direction)

In [75]:
def separate_dir(df):
    up_data = pd.DataFrame()
    down_data = pd.DataFrame()
    previous_bus_stop=df.iloc[0]['bus_stop']
    previous_bus = df.iloc[0]['device_id']
    if(previous_bus_stop>=len(b_stops)//4):
        up = False
        #df.iloc[0]['status']=0
    else:
        up = True
        #df = df.iloc[0]['status']=1
        
     
    for index,row in df.iterrows():
        current_bus_stop = row['bus_stop']
        current_bus = row['device_id']
        if(previous_bus!=current_bus):
            previous_bus_stop=row['bus_stop']
            previous_bus = row['device_id']
            if(previous_bus_stop>=len(b_stops)//4):
                up = False
        
            else:
                up = True
            
        
            
        if up:
            if(previous_bus_stop<=current_bus_stop):
                
                up_data=up_data.append(row)
            else:
                up = False
                
                down_data = down_data.append(row)
                
        else:
            if(previous_bus_stop>=current_bus_stop):
                
                down_data =down_data.append(row)
            else:
                up = True
                
                up_data =up_data.append(row)
        
        previous_bus_stop = current_bus_stop    
        
            
    
    return up_data,down_data       

In [2]:
# create training data for regression model

In [76]:
def create_pair(df,UP):
    pairs = pd.DataFrame()
    for i in range(df.shape[0]-1):
        prev = df.iloc[i]['bus_stop']
        current = df.iloc[i+1]['bus_stop']
        j=i+1
        
        if(UP==True):
            while current>prev and j<df.shape[0]:
                s = df.iloc[i]['bus_stop']
                d = df.iloc[j]['bus_stop']
                start_time = df.iloc[i]['date_time']
                route_id = df.iloc[i]['route_id']
                weekday = df.iloc[i]['weekday']
                travel_time= df.iloc[j]['date_time']-df.iloc[i]['date_time']
                record = pd.Series({'s':s,'d':d,'start_time':start_time,'route_id':route_id,'weekday':weekday,'travel_time':travel_time})
                #print(record)
                pairs = pairs.append(record,ignore_index=True)
                #print(df.iloc[i]['bus_stop'],df.iloc[j]['bus_stop'])
                j=j+1
                if(j<df.shape[0]-1):
                    prev = current
                    current = df.iloc[j+1]['bus_stop']
            if(j<df.shape[0]):
                if(df.iloc[i]['bus_stop'] < df.iloc[j]['bus_stop']):
                    record = pd.Series({'s':s,'d':d,'start_time':start_time,'route_id':route_id,'weekday':weekday,'travel_time':travel_time})
                    #print(df.iloc[i]['bus_stop'],df.iloc[j]['bus_stop'])
                    pairs = pairs.append(record,ignore_index=True)
        else:
            while current<prev and j<df.shape[0]:
                s = df.iloc[i]['bus_stop']
                d = df.iloc[j]['bus_stop']
                start_time = df.iloc[i]['date_time']
                route_id = df.iloc[i]['route_id']
                weekday = df.iloc[i]['weekday']
                travel_time= df.iloc[j]['date_time']-df.iloc[i]['date_time']
                record = pd.Series({'s':s,'d':d,'start_time':start_time,'route_id':route_id,'weekday':weekday,'travel_time':travel_time})
                #print(record)
                pairs = pairs.append(record,ignore_index=True)
                #print(df.iloc[i]['bus_stop'],df.iloc[j]['bus_stop'])
                j=j+1
                if(j<df.shape[0]-1):
                    prev = current
                    current = df.iloc[j+1]['bus_stop']
            if(j<df.shape[0]):
                if(df.iloc[i]['bus_stop'] > df.iloc[j]['bus_stop']):
                    record = pd.Series({'s':s,'d':d,'start_time':start_time,'route_id':route_id,'weekday':weekday,'travel_time':travel_time})
                    #print(df.iloc[i]['bus_stop'],df.iloc[j]['bus_stop'])
                    pairs = pairs.append(record,ignore_index=True)
    return  pairs    
        
    

In [ ]:
#append all data frame to create single data frame

In [77]:
def appenddata(s_lat,s_long,df,n,data):
    for index,line in data.iterrows():
        dist = vincenty((s_lat,s_long),(line['lat'],line['long'])).meters
        if dist < 50:
            line['b_distance']=dist
            line['bus_stop']=n
            df= df.append(line)        
        else:
            continue
    return df

In [78]:
with open('../K-4/june2_thursday/devices.txt') as file:
    devices = json.load(file)

In [79]:
devices

[150221241,
 150811705,
 150811496,
 150218048,
 150221473,
 150810425,
 150218772,
 150810849]

In [80]:
all_data = pd.DataFrame()

In [81]:
for d in devices:
    name = 'd_'+str(d)
    d_data = pd.read_csv('../K-4/june2_thursday/'+name)
    all_data = all_data.append(d_data)

In [82]:
all_data.to_csv('all_data')

In [83]:
with open('../K-4/june2_thursday/b_stops.txt') as file:
    b_stops = json.load(file)
    

In [84]:
all_data = pd.read_csv('2016-06-14')

In [85]:
all_data.head()

,Unnamed: 0,device_id,date_time,lat,long,real_time
0,205454,150810425,2016-06-14 07:54:44,12.985648,77.664009,2016-06-14 07:54:44
1,205455,150810425,2016-06-14 07:54:54,12.985648,77.664009,2016-06-14 07:54:54
2,205456,150221473,2016-06-14 07:55:02,12.980518,77.512222,2016-06-14 07:55:02
3,205457,150219191,2016-06-14 07:55:03,12.966298,77.606979,2016-06-14 07:55:03
4,205458,150810425,2016-06-14 07:55:04,12.985648,77.664009,2016-06-14 07:55:04


In [86]:
df = pd.DataFrame()

In [87]:
k=0
for i in range(0,len(b_stops),2):
    
    s_lat = b_stops[i]['latlons'][0]
    s_long = b_stops[i]['latlons'][1]
    k+=1
    print(i,k)
    df=appenddata(s_lat,s_long,df,k,all_data)

0 1
2 2
4 3
6 4
8 5
10 6
12 7
14 8
16 9
18 10
20 11
22 12
24 13
26 14
28 15
30 16
32 17
34 18
36 19
38 20


In [88]:
#df.to_csv('sorted_by_bus_stop_50_meter')

In [89]:
df.sort_values(by=['device_id','date_time'],inplace=True)

In [90]:
df.to_csv(name+'sorted_by_device_id_time')

In [91]:
devices = df.device_id.unique()

In [92]:
d_mapping = {}
i =1
for d in devices:
    d_mapping[d] = i
    i= i+1

In [93]:
df.replace({'device_id': d_mapping} , inplace=True)

In [94]:
df.to_csv(name+'mapped devices')

In [95]:
#df['bus_stop'] =df['bus_stop'].astype(int)

In [97]:
devices

array([  1.50218048e+08,   1.50219191e+08,   1.50221473e+08,
         1.50810425e+08])

In [98]:
df['date_time'] = df['date_time'].astype('str').map(lambda x : x.split()[1])

In [99]:
df['real_time'] = df['date_time']

In [100]:
df['date_time'] = df['date_time'].map(lambda x : get_hour_min(x))

In [101]:
df = df[['bus_stop','device_id','date_time','real_time']]

In [102]:
df.head()

,bus_stop,device_id,date_time,real_time
377,6.0,1.0,490,08:10:31
381,6.0,1.0,490,08:10:41
385,6.0,1.0,490,08:10:51
417,7.0,1.0,492,08:12:10
421,7.0,1.0,492,08:12:20


In [103]:
unique_data = remove_duplicates(df)

In [104]:
#unique_data[unique_data['device_id']==2]

In [105]:
#unique_data.to_csv('sorted_by_device_id_time_removed_duplicates')

In [109]:
up_data,down_data = separate_dir(unique_data)

In [110]:
down_data.to_csv('down_data'+name)

In [111]:
up_data.to_csv('up_data'+name)

In [70]:
down_data = pd.read_csv('down_data'+name)

In [71]:
up_data = pd.read_csv('up_data'+name)

In [260]:
down_data['weekday']=0

In [261]:
down_data['route_id']=1

In [262]:
up_data['route_id']=1

In [263]:
up_data['weekday']=0

In [264]:
pairs = create_pair(up_data,True)

In [231]:
pairs.to_csv('pairs_up'+name)

In [415]:
#headway_down_data = down_data.sort_values(by=['bus_stop','date_time'])

In [417]:
#headway_down_data.to_csv('headway_down_data')

In [421]:
def create_headway(df):
    headway_pair = pd.DataFrame()
    for i in range(df.shape[0]-1):
        row = df.iloc[i]
        next_row = df.iloc[i+1]
        if(row['bus_stop']==next_row['bus_stop']):
            record = pd.Series({'bus_stop':row['bus_stop'],'bus_1':row['device_id'],'bus2':next_row['device_id'],'headway':next_row['date_time']-row['date_time']})
            headway_pair = headway_pair.append(record,ignore_index=True)
        else:
            continue
    return headway_pair   
    

In [422]:
#headway_pair = create_headway(headway_down_data)

In [1]:
#headway_pair

In [1]:
i =0